<a href="https://colab.research.google.com/github/ofoofoo/dynamic-token-merging/blob/main/experiments/merge_sequence_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install ptflops
!pip install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename

In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
from tqdm import tqdm
from ptflops import get_model_complexity_info

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import random
import time
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
import torch
from fvcore.nn import FlopCountAnalysis

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
test_dataset = ds['test']
subset_size = 100
random_indices = random.sample(range(len(test_dataset)), subset_size)
test_dataset = test_dataset.select(random_indices)
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [11]:
total_flops = 0
total_time = 0
outputs = []
targets = []

In [4]:
print("Vocab Size", model.config.vocab_size)
print("Hidden Size", model.config.hidden_size)
print(model.lm_head)

Vocab Size 50264
Hidden Size 1024
Linear(in_features=1024, out_features=50264, bias=False)


In [ ]:
import torch.nn as nn
from fvcore.nn import FlopCountAnalysis
from transformers.modeling_outputs import BaseModelOutput

class PrunedEncoder(nn.Module):
    def __init__(self, encoder, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedEncoder, self).__init__()
        self.encoder = encoder
        self.num_pruning_steps = num_pruning_steps
        self.pruning_proportion = pruning_proportion

    def forward(self, input_ids):
        inputs_embeds = self.encoder.embed_tokens(input_ids)
        embed_pos = self.encoder.embed_positions(input_ids)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = self.encoder.layernorm_embedding(encoder_hidden_states)

        attention = None
        for i in range(self.num_pruning_steps):
            for j in range(12 // self.num_pruning_steps):
                current_layer_index = i * (12 // self.num_pruning_steps) + j
                layer_outputs = self.encoder.layers[current_layer_index](
                    encoder_hidden_states,
                    attention_mask=None,
                    layer_head_mask=None,
                    output_attentions=True
                )
                encoder_hidden_states, attention = layer_outputs[:2]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, self.pruning_proportion)
            mask = avg_attention > threshold
            mask = mask.squeeze(0) # [seq_len]
            #encoder_hidden_states[mask].unsqueeze(dim=0)
            encoder_hidden_states = encoder_hidden_states[:, mask, :]

        return encoder_hidden_states



class PrunedModel(nn.Module):
    def __init__(self, model, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedModel, self).__init__()
        self.encoder = PrunedEncoder(model.model.encoder, num_pruning_steps, pruning_proportion)
        self.decoder = model.model.decoder
        self.lm_head = model.lm_head
        self.config = model.config

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs,
            encoder_attention_mask=None,
            past_key_values=None,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


class ReferenceModel(nn.Module):
    def __init__(self, model):
        super(ReferenceModel, self).__init__()
        self.model = model

    def forward(self, input_ids, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=None,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        return outputs.logits

all_results = {}

for i in [0.2]:
  print(f"RUNNING WITH PRUNING PROPORTION: {i}")
  total_flops_pruned = 0
  total_flops_ref = 0
  total_time = 0
  total_time_ref = 0
  outputs = []
  outputs_ref = []
  targets = []

  pruned_model_instance = PrunedModel(model, pruning_proportion=i).to(device)
  reference_model_instance = ReferenceModel(model).to(device)

  for example in tqdm(test_dataset):
      article = example['article']
      reference_summary = example['highlights']
      tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
      token_ids = tokenizer.convert_tokens_to_ids(tokens)
      token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

      decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

      with torch.no_grad():
          print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
          model.eval()
          print('')

          # measure FLOPs: need to run this before timing
          flops_pruned = FlopCountAnalysis(
              pruned_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_pruned += flops_pruned

          # pruned model generation
          start_time = time.time()
          generated_tokens = model.generate(
              encoder_outputs=BaseModelOutput(
                  last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
              ),
              decoder_input_ids=decoder_input_ids,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          generated_summary = tokenizer.decode(
              generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time = time.time()
          total_time += (end_time - start_time)

          flops_ref = FlopCountAnalysis(
              reference_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_ref += flops_ref

          # reference model generation
          start_time_ref = time.time()
          real_generated_tokens = model.generate(
              input_ids=token_ids_tensor,
              attention_mask=None,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          real_generated_summary = tokenizer.decode(
              real_generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time_ref = time.time()
          total_time_ref += (end_time_ref - start_time_ref)

          print('')
          print(generated_summary)
          print('----------------')
          print(real_generated_summary)
          print('----------------')
          print(reference_summary)
          outputs.append(generated_summary)
          outputs_ref.append(real_generated_summary)
          targets.append(reference_summary)

  average_flops_pruned = total_flops_pruned / len(test_dataset)
  average_flops_ref = total_flops_ref / len(test_dataset)
  average_time = total_time / len(test_dataset)
  average_time_ref = total_time_ref / len(test_dataset)

  rouge_results = rouge.compute(predictions=outputs, references=targets)
  bleu_results = bleu.compute(predictions=outputs, references=targets)

  rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
  bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

  print("Inference Results:")
  print(f"Average forward pass time: {average_time:.4f} seconds")
  print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
  print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
  print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
  print("ROUGE Scores:", rouge_results)
  print("ROUGE Scores (reference):", rouge_results_ref)
  print("BLEU Score:", bleu_results)
  print("BLEU Score (reference):", bleu_results_ref)

  summary_stats = {"average_flops_pruned": average_flops_pruned,
      "average_flops_ref": average_flops_ref,
      "average_time_pruned": average_time,
      "average_time_ref": average_time_ref,
      "average_rouge_pruned": rouge_results,
      "average_rouge_ref": rouge_results_ref,
      "average_bleu_pruned": bleu_results,
      "average_bleu_ref": bleu_results_ref}
  all_results[i] = summary_stats

In [ ]:
# MERGING INSTEAD OF PRUNING

import torch
import torch.nn as nn
from fvcore.nn import FlopCountAnalysis
from transformers.modeling_outputs import BaseModelOutput
from tqdm import tqdm

class PrunedEncoder(nn.Module):
    def __init__(self, encoder, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedEncoder, self).__init__()
        self.encoder = encoder
        self.num_pruning_steps = num_pruning_steps
        self.pruning_proportion = pruning_proportion

    def forward(self, input_ids):
        inputs_embeds = self.encoder.embed_tokens(input_ids)
        embed_pos = self.encoder.embed_positions(input_ids)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = self.encoder.layernorm_embedding(encoder_hidden_states)

        attention = None
        for i in range(self.num_pruning_steps):
            for j in range(12 // self.num_pruning_steps):
                current_layer_index = i * (12 // self.num_pruning_steps) + j
                layer_outputs = self.encoder.layers[current_layer_index](
                    encoder_hidden_states,
                    attention_mask=None,
                    layer_head_mask=None,
                    output_attentions=True
                )
                encoder_hidden_states, attention = layer_outputs[:2]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, self.pruning_proportion)
            mask = avg_attention > threshold
            mask = mask.squeeze(0) # [seq_len]
            #encoder_hidden_states = encoder_hidden_states[:, mask, :]
            anchors = torch.where(mask)[0]  # positions of significant tokens
            new_hidden_states = []
            current_pos = 0
            seq_len = encoder_hidden_states.size(1)

            for anchor_pos in anchors:
                if current_pos < anchor_pos:
                    avg_token = encoder_hidden_states[:, current_pos:anchor_pos + 1, :].mean(dim=1, keepdim=True) # current_pos:anchor_pos + 1 MERGES ALL INSIGNIFICANT TOKENS TO THE ANCHOR TOKEN
                    new_hidden_states.append(avg_token)
                # for appending the anchor token
                anchor_token = encoder_hidden_states[:, anchor_pos:anchor_pos+1, :]
                new_hidden_states.append(anchor_token)
                current_pos = anchor_pos + 1

            if current_pos < seq_len:
                avg_token = encoder_hidden_states[:, current_pos:, :].mean(dim=1, keepdim=True)
                new_hidden_states.append(avg_token)
            encoder_hidden_states = torch.cat(new_hidden_states, dim=1)

        return encoder_hidden_states


class PrunedModel(nn.Module):
    def __init__(self, model, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedModel, self).__init__()
        self.encoder = PrunedEncoder(model.model.encoder, num_pruning_steps, pruning_proportion)
        self.decoder = model.model.decoder
        self.lm_head = model.lm_head
        self.config = model.config

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs,
            encoder_attention_mask=None,
            past_key_values=None,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


class ReferenceModel(nn.Module):
    def __init__(self, model):
        super(ReferenceModel, self).__init__()
        self.model = model

    def forward(self, input_ids, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=None,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        return outputs.logits

all_results = {}

for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
  print(f"RUNNING WITH PRUNING PROPORTION: {i}")
  total_flops_pruned = 0
  total_flops_ref = 0
  total_time = 0
  total_time_ref = 0
  outputs = []
  outputs_ref = []
  targets = []

  pruned_model_instance = PrunedModel(model, pruning_proportion=i).to(device)
  reference_model_instance = ReferenceModel(model).to(device)

  for example in tqdm(test_dataset):
      article = example['article']
      reference_summary = example['highlights']
      tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
      token_ids = tokenizer.convert_tokens_to_ids(tokens)
      token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

      decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

      with torch.no_grad():
          print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
          model.eval()
          print('')

          # measure FLOPs: need to run this before timing
          flops_pruned = FlopCountAnalysis(
              pruned_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_pruned += flops_pruned

          # pruned model generation
          start_time = time.time()
          generated_tokens = model.generate(
              encoder_outputs=BaseModelOutput(
                  last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
              ),
              decoder_input_ids=decoder_input_ids,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          generated_summary = tokenizer.decode(
              generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time = time.time()
          total_time += (end_time - start_time)

          flops_ref = FlopCountAnalysis(
              reference_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_ref += flops_ref

          # reference model generation
          start_time_ref = time.time()
          real_generated_tokens = model.generate(
              input_ids=token_ids_tensor,
              attention_mask=None,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          real_generated_summary = tokenizer.decode(
              real_generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time_ref = time.time()
          total_time_ref += (end_time_ref - start_time_ref)

          print('')
          print(generated_summary)
          print('----------------')
          print(real_generated_summary)
          print('----------------')
          print(reference_summary)
          outputs.append(generated_summary)
          outputs_ref.append(real_generated_summary)
          targets.append(reference_summary)

  average_flops_pruned = total_flops_pruned / len(test_dataset)
  average_flops_ref = total_flops_ref / len(test_dataset)
  average_time = total_time / len(test_dataset)
  average_time_ref = total_time_ref / len(test_dataset)

  rouge_results = rouge.compute(predictions=outputs, references=targets)
  bleu_results = bleu.compute(predictions=outputs, references=targets)

  rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
  bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

  print("Inference Results:")
  print(f"Average forward pass time: {average_time:.4f} seconds")
  print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
  print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
  print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
  print("ROUGE Scores:", rouge_results)
  print("ROUGE Scores (reference):", rouge_results_ref)
  print("BLEU Score:", bleu_results)
  print("BLEU Score (reference):", bleu_results_ref)

  summary_stats = {"average_flops_pruned": average_flops_pruned,
      "average_flops_ref": average_flops_ref,
      "average_time_pruned": average_time,
      "average_time_ref": average_time_ref,
      "average_rouge_pruned": rouge_results,
      "average_rouge_ref": rouge_results_ref,
      "average_bleu_pruned": bleu_results,
      "average_bleu_ref": bleu_results_ref}
  all_results[i] = summary_stats